# "Intuitively understanding Python decorators"
> "Syntactic sugar and functional implementation of decorators"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [Python, functional programming]
- image: images/decorators/decorator.jpg
- hide: false
- search_exclude: true

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

## Functions
Before you can understand decorators, you must first understand how functions work. For our purposes, a function returns a value based on the given arguments. Here is a very simple example:

In [9]:
def add_one(number): return number+1

add_one(2)

3

In general, functions in Python may also have side effects rather than just turning an input into an output. The `print()` function is a basic example of this: it returns `None` while having the side effect of outputting something to the console. However, to understand decorators, it is enough to think about functions as something that turns given arguments into a value.

### First-class objects
In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (`string`, `int`, `float`, `list`, and so on). Consider the following three functions:

In [10]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, `say_hello()` and `be_awesome()` are regular functions that expect a name given as a string. The `greet_bob()` function however, expects a function as its argument. We can, for instance, pass it the `say_hello()` or the `be_awesome()` function:

In [11]:
greet_bob(say_hello)

'Hello Bob'

In [12]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

Note that `greet_bob(say_hello)` refers to two functions, but in different ways: `greet_bob()` and `say_hello`. The `say_hello` function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The `greet_bob()` function, on the other hand, is written with parentheses, so it will be called as usual.

### Inner-functions
It’s possible to define functions inside other functions. Such functions are called _inner functions_. Here’s an example of a function with two inner functions:

In [13]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

What happens when you call the `parent()` function? Think about this for a minute. The output will be as follows:

In [14]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Note that the order in which the inner functions are defined does not matter. Like with any other functions, the printing only happens when the inner functions are executed.

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to `parent()`: they only exist inside the `parent()` function as local variables. Try calling `first_child()`. You should get an error:

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to `parent()`: they only exist inside the `parent()` function as local variables. Try calling `first_child()`. You should get an error:

In [15]:
# collapse
first_child()

NameError: name 'first_child' is not defined

Whenever you call `parent()`, the inner functions `first_child()` and `second_child()` are also called. But because of their local scope, they aren’t available outside of the `parent()` function.

### Returning functions from functions
Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer `parent()` function:

In [16]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

Note that you are returning `first_child` without the parentheses. Recall that this means that you are returning a reference to the function `first_child`. In contrast `first_child()` with parentheses refers to the result of evaluating the function. This can be seen in the following example:

In [17]:
first = parent(1)
second = parent(2)

In [18]:
first

<function __main__.parent.<locals>.first_child()>

In [19]:
second

<function __main__.parent.<locals>.second_child()>

The somewhat cryptic output simply means that the first variable refers to the local `first_child()` function inside of `parent()`, while second points to `second_child()`.

You can now use first and second as if they are regular functions, even though the functions they point to can’t be accessed directly:

In [20]:
first()

'Hi, I am Emma'

In [21]:
second()

'Call me Liam'

Finally, note that in the earlier example you executed the inner functions within the parent function, for instance `first_child()`. However, in this last example, you did not add parentheses to the inner functions — `first_child` — upon returning. That way, you got a reference to each function that you could call in the future. Make sense?

## Simple decorators
Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [22]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [23]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


To understand what’s going on here, look back at the previous examples. We are literally just applying everything you have learned so far.

The so-called decoration happens at the following line:
```python
say_whee = my_decorator(say_whee)
```

In effect, the name `say_whee` now points to the `wrapper()` inner function. Remember that you return wrapper as a function when you call `my_decorator(say_whee)`:

In [24]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

However, `wrapper()` has a reference to the original `say_whee()` as func, and calls that function between the two calls to `print()`.

Put simply: __decorators wrap a function, modifying its behavior__.

Before moving on, let’s have a look at a second example. Because `wrapper()` is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [25]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

If you try to call `say_whee()` after bedtime, nothing will happen:

## Syntactic sugar
The way you decorated `say_whee()` above is a little clunky. First of all, you end up typing the name `say_whee` three times. In addition, the decoration gets a bit hidden away below the definition of the function.

Instead, Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the “pie” syntax. The following example does the exact same thing as the first decorator example:

In [26]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

So, `@my_decorator` is just an easier way of saying `say_whee = my_decorator(say_whee)`. It’s how you apply a decorator to a function.

### Reusing decorators
Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

Create a file called `decorators.py` with the following content:

In [27]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

> Note: You can name your inner function whatever you want, and a generic name like `wrapper()` is usually okay. You’ll see a lot of decorators in this article. To keep them apart, we’ll name the inner function with the same name as the decorator but with a `wrapper_ prefix`.

You can now use this new decorator in other files by doing a regular import:

```python
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")
```

When you run this example, you should see that the original say_whee() is executed twice:

```python
>>> say_whee()
Whee!
Whee!
```

### Decorating functions with arguments
Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:

In [28]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

Unfortunately, running this code raises an error:

In [29]:
# collapse
greet("World")

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function `wrapper_do_twice()` does not take any arguments, but `name="World"` was passed to it. You could fix this by letting `wrapper_do_twice()` accept one argument, but then it would not work for the `say_whee()` function you created earlier.

The solution is to use `*args` and `**kwargs` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite `do_twice()` as follows:

In [30]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

The `wrapper_do_twice()` inner function now accepts any number of arguments and passes them on to the function it decorates. Now both your `say_whee()` and `greet()` examples works:

In [31]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [32]:
@do_twice
def greet(name):
    print(f"Hello {name}")

greet("World")

Hello World
Hello World


### Returning values from decorated functions
What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [33]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [34]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


In [35]:
print(hi_adam)

None


Oops, your decorator ate the return value from the function.

Because the `do_twice_wrapper()` doesn’t explicitly return a value, the call `return_greeting("Adam")` ended up returning `None`.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorator:

In [40]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

The return value from the last execution of the function is returned:

In [42]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

return_greeting("Adam")

Creating greeting
Creating greeting


'Hi Adam'

## Some examples
Let’s look at a few more useful examples of decorators. You’ll notice that they’ll mainly follow the same pattern that you’ve learned so far:

```python
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator
```

This formula is a good boilerplate template for building more complex decorators.

### Timing decorator
Let’s start by creating a `@timer` decorator. It will measure the time a function takes to execute and print the duration to the console. Here’s the code:

In [44]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      
        run_time = end_time - start_time    
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

## Writing a custom transform in `fastai` with decorators
If you want to write a custom transform to apply to your data, the easiest way is to write a function. A `Transform` will be applied only to a matching type, if a type is provided (otherwise, it will always be applied). In the following code, the `:int` in the function signature means that `f` gets applied only to `ints`. That's why `tfm(2.0)` returns `2.0`, but `tfm(2)` returns `3` here:

In [ ]:
from fastai.text.all import *

def f(x:int): return x+1
tfm = Transform(f)
tfm(2), tfm(2.0)